# Implementation to consider sample intervals

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.visual import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon.lexicon_memoris import LEG_CONTAMINATION_LEV
from utils.config import DEFAULT_LITHO_LEXICON, DEFAULT_LITHO_LEGEND, DEFAULT_POL_LEXICON
from utils.io import dataframe_viewer, dict_viewer
from utils.orm import boreholes_from_dataframe
from utils.visual import legend_from_attributes
from core.project import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
df = pd.read_csv('real_data_subset_for_test.csv', sep=',')

In [3]:
dataframe_viewer(df, rows=5, un_val='ID')

Rows : 112, columns : 19, Unique values on col 'ID': 46


interactive(children=(IntSlider(value=5, description='rows', max=112, min=5, readout=False), IntSlider(value=1…

In [4]:
df_litho = df[['ID', 'X', 'Y', 'Z', 'Long_for', 'Litho_top', 'Litho_base', 'Epais_intv', 'Lithology']]
df_litho.rename(columns={'Lithology':'Description'}, inplace=True)

df_ech = df[['ID', 'X', 'Y', 'Z', 'Long_for', 'ID_ech', 'Type_ech', 'Top_intv', 'Base_intv', 
             'Epais_intv', 'Bnz', 'Toln', 'Pb', 'As', 'Cu', 'Ni']]
df_ech.rename(columns={'Top_intv':'Ech_top', 'Base_intv':'Ech_base'}, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [5]:
pollutants = []
for i, c in enumerate(df.columns):
    if c in DEFAULT_POL_LEXICON.abbreviations.keys() or c in DEFAULT_POL_LEXICON.abbreviations.values():
        print(f"{i}: {c}")
        pollutants.append(c)

13: Bnz
14: Toln
15: Pb
16: As
17: Cu
18: Ni


In [6]:
bh_id='201'
dataframe_viewer(df_litho.query(f"ID=='{bh_id}'"), rows=3, cols=16), dataframe_viewer(df_ech.query(f"ID=='{bh_id}'"), rows=3, cols=16)

Rows : 2, columns : 9


interactive(children=(IntSlider(value=2, description='rows', max=2, min=2, readout=False), IntSlider(value=9, …

Rows : 2, columns : 16


interactive(children=(IntSlider(value=2, description='rows', max=2, min=2, readout=False), IntSlider(value=16,…

(None, None)

In [7]:
# adds a description in sample (TEST)
df_ech['Description'] = np.nan
df_ech.loc[:1, 'Description'] = ['Sable', 'Limon']

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


## Create Boreholes from dataframes

In [8]:
data_dict = {'lithologies_data': df_litho, 'pollutants_data': df_ech}

In [9]:
boreholes, components, link_intv_comp = boreholes_from_dataframe(data_dict, verbose=False,
                                                sample_type_col='Type_ech', diameter_col='Diam_for', 
                                                average_z=102, attributes=['lithology']+pollutants, 
                                                symbols={'lithology':{'lexicon':DEFAULT_LITHO_LEXICON}})


Data Processing...
To continue, default diameter column has been created with value: 0.1 [m]

 BH_ID: '201'
0- Interval top=0.6, base=1.2, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

1- Interval top=0.6, base=1.2, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

2- Interval top=0.7, base=1.2, type=sample
 - Interval components: [Component({'lithology': 'sable'}), Component({'benzène': 'vs'}), Component({'toluène': 'vr'}), Component({'plomb': 'vr'}), Component({'arsenic': 'inconnu'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

3- Interval top=2.0, base=2.4, type=sample
 - Interval components: [Component({'lithology': 'limon'}), Component({'benzène': 'vr'}), Component({'toluène': 'inconnu'}), Component({'plomb': 'vs'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

 Summary : {'201': {'lithology': Striplog(2 Intervals, start=0.6, stop=1.2), 'sample': Striplog(2 Int

/home/yanathan/Projects/GSDMA/utils/orm.py:202: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="components"' to the 'ComponentOrm.intervals' relationship.
  boreholes_orm.append(BoreholeOrm(id=bh_name, date=bh_date))
/home/yanathan/Projects/GSDMA/utils/orm.py:202: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.intv_id, which conflic


 BH_ID: '205'
0- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'})]

1- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'})]

2- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'})]

3- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'})]

4- Interval top=0.2, base=0.6, type=sample
 - Interval components: [Component({'benzène': 'vs'}), Component({'toluène': 'vr'}), Component({'plomb': 'vr'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vs'})]

5- Interval top=1.9, base=2.4, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vr'}), Component({'plomb': 'vs'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

6- Interval top=3.0, base=3.2, type=sample
 - Interval components: [Com

sample_intv: {25: {'interval_number': 5, 'top': <core.orm.PositionOrm object at 0x7f57bf7cc490>, 'base': <core.orm.PositionOrm object at 0x7f57bf7cc910>, 'type': 'sample', 'description': '{"benz\\u00e8ne": "vs"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vs"}, {"arsenic": "vr"}, {"cuivre": "vr"}, {"nickel": "vr"}'}, 26: {'interval_number': 6, 'top': <core.orm.PositionOrm object at 0x7f57bf7cc8b0>, 'base': <core.orm.PositionOrm object at 0x7f57bf834fd0>, 'type': 'sample', 'description': '{"benz\\u00e8ne": "vs"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "vr"}, {"cuivre": "vr"}, {"nickel": "vr"}'}, 27: {'interval_number': 7, 'top': <core.orm.PositionOrm object at 0x7f57bf834ee0>, 'base': <core.orm.PositionOrm object at 0x7f57bf8349a0>, 'type': 'sample', 'description': '{"benz\\u00e8ne": "vi"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vi"}, {"arsenic": "inconnu"}, {"cuivre": "vr"}, {"nickel": "vr"}'}, 28: {'interval_number': 8, 'top': <core.orm.PositionOrm object at 0x7f57bf834490>, 'bas

sample_intv: {50: {'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f57bf83bcd0>, 'base': <core.orm.PositionOrm object at 0x7f57bf9304f0>, 'type': 'sample', 'description': '{"benz\\u00e8ne": "vi"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "vr"}, {"cuivre": "vr"}, {"nickel": "vr"}'}, 51: {'interval_number': 5, 'top': <core.orm.PositionOrm object at 0x7f57bf930d00>, 'base': <core.orm.PositionOrm object at 0x7f57bf9300a0>, 'type': 'sample', 'description': '{"benz\\u00e8ne": "vi"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "vs"}, {"cuivre": "vr"}, {"nickel": "vr"}'}, 52: {'interval_number': 6, 'top': <core.orm.PositionOrm object at 0x7f57bf9305e0>, 'base': <core.orm.PositionOrm object at 0x7f57bf930be0>, 'type': 'sample', 'description': '{"benz\\u00e8ne": "vs"}, {"tolu\\u00e8ne": "inconnu"}, {"plomb": "vr"}, {"arsenic": "inconnu"}, {"cuivre": "inconnu"}, {"nickel": "inconnu"}'}, 53: {'interval_number': 7, 'top': <core.orm.PositionOrm object at 0x7f57

 BH_ID: '226'
0- Interval top=0.0, base=1.0, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

1- Interval top=0.0, base=1.0, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

2- Interval top=0.0, base=1.0, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

3- Interval top=0.0, base=1.0, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

4- Interval top=0.6, base=1.0, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vr'}), Component({'plomb': 'vr'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

5- Interval top=2.1, base=2.4, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vr'}), Component({'plomb': 'vr'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

6- Interval top=2.8, base=3.2, type=sample
 - Interval c

litho_intv: {138: {'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f57bf7f6190>, 'base': <core.orm.PositionOrm object at 0x7f57bf7eb9a0>, 'type': 'lithology', 'description': '{"lithology": "limon"}'}}
sample_intv: {139: {'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f57bf745e20>, 'base': <core.orm.PositionOrm object at 0x7f57bf745610>, 'type': 'sample', 'description': '{"benz\\u00e8ne": "vi"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "inconnu"}, {"arsenic": "inconnu"}, {"cuivre": "inconnu"}, {"nickel": "inconnu"}'}}

 BH_ID: 'F5M'
0- Interval top=0.0, base=0.5, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

1- Interval top=0.0, base=0.5, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

2- Interval top=0.0, base=0.5, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

3- Interval top=0.0, base=1.2, type=lithology
 - Interval components: [Component({'lithology': 'remblais'

 BH_ID: 'F16M'
0- Interval top=0.0, base=3.6, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

1- Interval top=0.7, base=1.2, type=sample
 - Interval components: [Component({'benzène': 'inconnu'}), Component({'toluène': 'inconnu'}), Component({'plomb': 'vr'}), Component({'arsenic': 'vs'}), Component({'cuivre': 'vs'}), Component({'nickel': 'vr'})]

 Summary : {'F16M': {'lithology': Striplog(1 Intervals, start=0.0, stop=3.6), 'sample': Striplog(1 Intervals, start=0.7, stop=1.2)}}
litho_intv: {174: {'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f57bf748b20>, 'base': <core.orm.PositionOrm object at 0x7f57bf748e50>, 'type': 'lithology', 'description': '{"lithology": "remblais"}'}}
sample_intv: {175: {'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f57bf757250>, 'base': <core.orm.PositionOrm object at 0x7f57bf7b5430>, 'type': 'sample', 'description': '{"benz\\u00e8ne": "inconnu"}, {"tolu\\u00e8ne": "inconnu"}, {"plomb": "vr"}, {

 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vr'}), Component({'plomb': 'vr'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

4- Interval top=3.0, base=3.5, type=sample
 - Interval components: [Component({'benzène': 'vs'}), Component({'toluène': 'inconnu'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

5- Interval top=5.2, base=5.7, type=sample
 - Interval components: [Component({'benzène': 'inconnu'}), Component({'toluène': 'inconnu'}), Component({'plomb': 'inconnu'}), Component({'arsenic': 'inconnu'}), Component({'cuivre': 'inconnu'}), Component({'nickel': 'inconnu'})]


 Summary : {'F25': {'sample': Striplog(3 Intervals, start=0.0, stop=5.7)}}
sample_intv: {189: {'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f57bf70d1f0>, 'base': <core.orm.PositionOrm object at 0x7f57bf70d4f0>, 'type': 'sample', 'description': '{"benz\\

In [10]:
len(boreholes), len(components)

(46, 29)

In [11]:
boreholes[0].litho_intv_values

{2: '{"lithology": "sable"}, {"benz\\u00e8ne": "vs"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "inconnu"}, {"cuivre": "vr"}, {"nickel": "vr"}', 3: '{"lithology": "limon"}, {"benz\\u00e8ne": "vr"}, {"tolu\\u00e8ne": "inconnu"}, {"plomb": "vs"}, {"arsenic": "vr"}, {"cuivre": "vr"}, {"nickel": "vr"}'}

In [12]:
boreholes[0].sample_intv_values

{2: '{"lithology": "sable"}, {"benz\\u00e8ne": "vs"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "inconnu"}, {"cuivre": "vr"}, {"nickel": "vr"}', 3: '{"lithology": "limon"}, {"benz\\u00e8ne": "vr"}, {"tolu\\u00e8ne": "inconnu"}, {"plomb": "vs"}, {"arsenic": "vr"}, {"cuivre": "vr"}, {"nickel": "vr"}'}

In [13]:
test = {'litho':{0:{'value':'sable'}}}
test
s = lambda k, v: test[k][v]
s('litho', 0)

{'value': 'sable'}

In [14]:
boreholes

[<core.orm.BoreholeOrm>(Name=201, Length=1.7999999999999998, Diameter=0.1, Intervals=2),
 <core.orm.BoreholeOrm>(Name=205, Length=3.0, Diameter=0.1, Intervals=4),
 <core.orm.BoreholeOrm>(Name=208, Length=2.999999999999999, Diameter=0.1, Intervals=4),
 <core.orm.BoreholeOrm>(Name=207, Length=3.600000000000001, Diameter=0.1, Intervals=5),
 <core.orm.BoreholeOrm>(Name=212, Length=2.9999999999999982, Diameter=0.1, Intervals=4),
 <core.orm.BoreholeOrm>(Name=214, Length=3.399999999999999, Diameter=0.1, Intervals=4),
 <core.orm.BoreholeOrm>(Name=217, Length=3.000000000000001, Diameter=0.1, Intervals=4),
 <core.orm.BoreholeOrm>(Name=221, Length=0.8, Diameter=0.1, Intervals=1),
 <core.orm.BoreholeOrm>(Name=223, Length=1.6, Diameter=0.1, Intervals=2),
 <core.orm.BoreholeOrm>(Name=?2, Length=0.7999999999999998, Diameter=0.1, Intervals=1),
 <core.orm.BoreholeOrm>(Name=?1, Length=0.7999999999999998, Diameter=0.1, Intervals=1),
 <core.orm.BoreholeOrm>(Name=224, Length=1.0, Diameter=0.1, Intervals=1)

In [15]:
components

{0: Component({'lithology': 'remblai'}),
 1: Component({'lithology': 'sable'}),
 2: Component({'benzène': 'vs'}),
 3: Component({'toluène': 'vr'}),
 4: Component({'plomb': 'vr'}),
 5: Component({'arsenic': 'inconnu'}),
 6: Component({'cuivre': 'vr'}),
 7: Component({'nickel': 'vr'}),
 8: Component({'lithology': 'limon'}),
 9: Component({'benzène': 'vr'}),
 10: Component({'toluène': 'inconnu'}),
 11: Component({'plomb': 'vs'}),
 12: Component({'arsenic': 'vr'}),
 13: Component({'lithology': 'boue'}),
 14: Component({'nickel': 'vs'}),
 15: Component({'benzène': 'vi'}),
 16: Component({'cuivre': 'inconnu'}),
 17: Component({'nickel': 'inconnu'}),
 18: Component({'plomb': 'inconnu'}),
 19: Component({'arsenic': 'vs'}),
 20: Component({'plomb': 'vi'}),
 21: Component({'toluène': 'vs'}),
 22: Component({'benzène': 'inconnu'}),
 23: Component({'lithology': 'schiste'}),
 24: Component({'cuivre': 'vs'}),
 25: Component({'nickel': 'vi'}),
 26: Component({'lithology': 'béton'}),
 27: Component({'

#  ---------------------------------- Test zone -----------------------------------------------

In [16]:
from striplog import Component, Striplog, Interval, Position
import re

In [17]:
pos = [Position(middle=0, x=125500, y=122500), # 0 
       Position(middle=0.5, x=125500, y=122500), # 1
       Position(middle=0.7, x=125500, y=122500), # 2
       Position(middle=1, x=125500, y=122500), # 3
       Position(middle=1.5, x=125500, y=122500), # 4
       
       Position(middle=0, x=126500, y=123500), # 5
       Position(middle=0.5, x=126500, y=123500), # 6
       Position(middle=0.7, x=126500, y=123500), # 7
       Position(middle=1, x=126500, y=123500), # 8
       Position(middle=1.5, x=126500, y=123500),] # 9

In [18]:
smp = [Component({'lithology': 'remblais'}),
           Component({'lithology': 'limon'}),
           Component({'lithology': 'schistes'}),
           Component({'lithology': 'sable'}),
           Component({'lithology': 'calcaire'}),
           Component({'lithology': 'inconnu'})]

In [19]:
t_A, b_A = 0, 3
t_B, b_B = 6, 7
row = 1
itv_A = Interval(top=pos[t_A], base=pos[b_A], components=[smp[0]], 
             data={'sample_type':'soil', 'litho_intv':{'itv_id':f'itv_{row}', 'value':'remblais', 'top':1, 'base':2}})
itv_B = Interval(top=pos[t_B], base=pos[b_B], components=[smp[1]], 
             data={'sample_type':'soil', 'litho_intv':{'itv_id':f'itv_{row+1}', 'value':'limon', 'top':2, 'base':3}})

print(itv_A, '\n\n', itv_B) 
print('\n', itv_A.relationship(itv_B),' --- ',  itv_B.relationship(itv_A))

{'top': Position({'middle': 0.0, 'upper': 0.0, 'lower': 0.0, 'x': 125500, 'y': 122500, 'units': 'm'}), 'base': Position({'middle': 1.0, 'x': 125500, 'y': 122500, 'units': 'm'}), 'description': '', 'data': {'sample_type': 'soil', 'litho_intv': {'itv_id': 'itv_1', 'value': 'remblais', 'top': 1, 'base': 2}}, 'components': [Component({'lithology': 'remblais'})]} 

 {'top': Position({'middle': 0.5, 'x': 126500, 'y': 123500, 'units': 'm'}), 'base': Position({'middle': 0.7, 'x': 126500, 'y': 123500, 'units': 'm'}), 'description': '', 'data': {'sample_type': 'soil', 'litho_intv': {'itv_id': 'itv_2', 'value': 'limon', 'top': 2, 'base': 3}}, 'components': [Component({'lithology': 'limon'})]}

 contains  ---  containedby


# -------------------------------------------------------------

## Create a dictionary containing legends for each representation attribute

In [20]:
legend_dict = legend_from_attributes([('lithology', DEFAULT_LITHO_LEGEND)]+pollutants)

In [21]:
dict_viewer(legend_dict)

lithology,"{'legend': Legend(Decor({'_colour': '#ffffe9', 'width': None, 'hatch': None, 'component': Component({'lithology': 'matériau(?:x)? meuble(?:s)?'})}) Decor({'_colour': '#fff497', 'width': None, 'hatch': ""'....'"", 'component': Component({'lithology': 'alluvion'})}) Decor({'_colour': '#b54500', 'width': None, 'hatch': None, 'component': Component({'lithology': 'boue'})}) Decor({'_colour': '#d3b798', 'width': None, 'hatch': ""'v'"", 'component': Component({'lithology': 'remblai'})}) Decor({'_colour': '#a5c7c9', 'width': None, 'hatch': ""'t'"", 'component': Component({'lithology': 'b[é|e]ton'})}) Decor({'_colour': '#8da3c9', 'width': None, 'hatch': ""'t'"", 'component': Component({'lithology': 'scorie'})}) Decor({'_colour': '#ffcc99', 'width': None, 'hatch': None, 'component': Component({'lithology': 'tourbe'})}) Decor({'_colour': '#ffeaa7', 'width': None, 'hatch': None, 'component': Component({'lithology': 'gypse'})}) Decor({'_colour': '#00151a', 'width': None, 'hatch': None, 'component': Component({'lithology': 'houille'})}) Decor({'_colour': '#798732', 'width': None, 'hatch': ""'-.'"", 'component': Component({'lithology': 'limon'})}) Decor({'_colour': '#ffcb23', 'width': None, 'hatch': ""'..'"", 'component': Component({'lithology': 'sable'})}) Decor({'_colour': '#adb7cc', 'width': None, 'hatch': ""'oo"", 'component': Component({'lithology': 'gravier'})}) Decor({'_colour': '#ace4c8', 'width': None, 'hatch': None, 'component': Component({'lithology': 'silt'})}) Decor({'_colour': '#d5e6cc', 'width': None, 'hatch': ""'---'"", 'component': Component({'lithology': 'argile'})}) Decor({'_colour': '#92dcb7', 'width': None, 'hatch': None, 'component': Component({'lithology': 'bentonite'})}) Decor({'_colour': '#bbffdd', 'width': None, 'hatch': ""'--'"", 'component': Component({'lithology': 'schiste'})}) Decor({'_colour': '#95ffca', 'width': None, 'hatch': None, 'component': Component({'lithology': 'argilite'})}) Decor({'_colour': '#d6fe9a', 'width': None, 'hatch': None, 'component': Component({'lithology': 'siltite'})}) Decor({'_colour': '#e1f0d8', 'width': None, 'hatch': None, 'component': Component({'lithology': 'tuffeau'})}) Decor({'_colour': '#69cf9c', 'width': None, 'hatch': None, 'component': Component({'lithology': 'silex'})}) Decor({'_colour': '#b7d9cc', 'width': None, 'hatch': None, 'component': Component({'lithology': 'conglomérat'})}) Decor({'_colour': '#019ccd', 'width': None, 'hatch': None, 'component': Component({'lithology': 'carbonate'})}) Decor({'_colour': '#149eff', 'width': None, 'hatch': ""'='"", 'component': Component({'lithology': 'calcaire'})}) Decor({'_colour': '#fdaaff', 'width': None, 'hatch': None, 'component': Component({'lithology': 'dolomite'})}) Decor({'_colour': '#fdaffe', 'width': None, 'hatch': None, 'component': Component({'lithology': 'dolomie'})}) Decor({'_colour': '#deeffe', 'width': None, 'hatch': None, 'component': Component({'lithology': 'craie'})}) Decor({'_colour': '#aac2c8', 'width': None, 'hatch': None, 'component': Component({'lithology': 'chert'})}) Decor({'_colour': '#000000', 'width': None, 'hatch': None, 'component': Component({'lithology': 'charbon'})}) Decor({'_colour': '#7ba1a8', 'width': None, 'hatch': None, 'component': Component({'lithology': 'marne'})}) Decor({'_colour': '#ffffff', 'width': None, 'hatch': None, 'component': Component({'lithology': 'inconnu'})}) Decor({'_colour': '#ffffff', 'width': None, 'hatch': None, 'component': Component({'lithology': 'not_exist'})}))}"
benzène,"{'legend': Legend(Decor({'_colour': '#00ff00', 'width': None, 'component': Component({'benzène': 'vr'}), 'hatch': None}) Decor({'_colour': '#ffa500', 'width': None, 'component': Component({'benzène': 'vs'}), 'hatch': None}) Decor({'_colour': '#ff0000', 'width': None, 'component': Component({'benzène': 'vi'}), 'hatch': None}) Decor({'_colour': '#ffffff', 'width': None, 'component': Component({'benzène': 'inconnu'}), 'hatch': None}))}"
toluène,"{'legend': Legend(Decor({'_colour': '#00ff00', 'wid

## Create a project and save data in a database

In [22]:
DB_name = 'tmp_files/real_orm_db.db'

if os.path.exists(DB_name):
    os.remove(DB_name)

engine = create_engine(f"sqlite:///{DB_name}", echo=True)

In [23]:
Base.metadata.create_all(engine)

2021-10-29 15:17:55,745 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-29 15:17:55,746 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-10-29 15:17:55,747 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-29 15:17:55,748 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-10-29 15:17:55,749 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-29 15:17:55,750 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Intervals")
2021-10-29 15:17:55,751 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-29 15:17:55,752 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Intervals")
2021-10-29 15:17:55,752 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-29 15:17:55,753 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Components")
2021-10-29 15:17:55,754 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-29 15:17:55,755 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Components")
2021-10-29 15:17:55,756 INFO sqlalchemy.engine.Engin

In [24]:
Session = sessionmaker(bind=engine)
session = Session()

#### Set project's legend dictionary

In [25]:
p = Project(session, name='Memoris_project', legend_dict=legend_dict, lexicon=DEFAULT_LITHO_LEXICON)

2021-10-29 15:17:55,821 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-29 15:17:55,823 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-10-29 15:17:55,823 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ()


#### Add components, boreholes into the project

In [26]:
p.add_components(components)

2021-10-29 15:17:55,831 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-10-29 15:17:55,833 INFO sqlalchemy.engine.Engine [generated in 0.00152s] ((0, "{'lithology': 'remblai'}"), (1, "{'lithology': 'sable'}"), (2, "{'benzène': 'vs'}"), (3, "{'toluène': 'vr'}"), (4, "{'plomb': 'vr'}"), (5, "{'arsenic': 'inconnu'}"), (6, "{'cuivre': 'vr'}"), (7, "{'nickel': 'vr'}")  ... displaying 10 of 29 total bound parameter sets ...  (27, "{'lithology': 'alluvion'}"), (28, "{'cuivre': 'vi'}"))
2021-10-29 15:17:55,835 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  0
2021-10-29 15:17:55,840 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-29 15:17:55,841 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-10-29 15:17:55,841 INFO sqlalchemy.engine.Engine [cached since 0.01

In [27]:
for bh in boreholes:
    p.add_borehole(bh)

2021-10-29 15:17:55,849 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, date, length, diameter) VALUES (?, ?, ?, ?)
2021-10-29 15:17:55,850 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ('201', None, 1.7999999999999998, 0.1)
2021-10-29 15:17:55,852 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-10-29 15:17:55,852 INFO sqlalchemy.engine.Engine [generated in 0.00048s] ((4, 101.226886, 101.226886, 101.226886, 152890.245758, 122596.47426), (5, 100.726886, 100.726886, 100.726886, 152890.245758, 122596.47426), (6, 99.926886, 99.926886, 99.926886, 152890.245758, 122596.47426), (7, 99.52688599999999, 99.52688599999999, 99.52688599999999, 152890.245758, 122596.47426))
2021-10-29 15:17:55,854 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, type, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-10-29 15:17:55,854 INFO sqlalchemy.engine.Engine [

2021-10-29 15:17:55,916 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-10-29 15:17:55,917 INFO sqlalchemy.engine.Engine [cached since 0.09453s ago] ()
2021-10-29 15:17:55,919 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".type AS "Intervals_type", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id", "Intervals".data_id AS "Intervals_data_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-10-29 15:17:55,920 INFO sqlalchemy.engine.Engine [cached since 0.05104s ago] ('205',)
2021-10-29 15:17:55,922 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS 

2021-10-29 15:17:55,963 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, type, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-10-29 15:17:55,964 INFO sqlalchemy.engine.Engine [cached since 0.1101s ago] ((16, '208', 4, 'sample', '{"benz\\u00e8ne": "vi"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "vs"}, {"cuivre": "vr"}, {"nickel": "vr"}', 32, 33, None), (17, '208', 5, 'sample', '{"benz\\u00e8ne": "vi"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "vr"}, {"cuivre": "vr"}, {"nickel": "vr"}', 34, 35, None), (18, '208', 6, 'sample', '{"benz\\u00e8ne": "vs"}, {"tolu\\u00e8ne": "inconnu"}, {"plomb": "vr"}, {"arsenic": "vr"}, {"cuivre": "vr"}, {"nickel": "vr"}', 36, 37, None), (19, '208', 7, 'sample', '{"benz\\u00e8ne": "vs"}, {"tolu\\u00e8ne": "inconnu"}, {"plomb": "inconnu"}, {"arsenic": "vr"}, {"cuivre": "inconnu"}, {"nickel": "vr"}', 38, 39, None))
2021-10-29 15:17:55,965 INFO sqlalchemy.engine.Engine C

Borehole geometry created successfully !
2021-10-29 15:17:56,014 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, date, length, diameter) VALUES (?, ?, ?, ?)
2021-10-29 15:17:56,016 INFO sqlalchemy.engine.Engine [cached since 0.1665s ago] ('207', None, 3.600000000000001, 0.1)
2021-10-29 15:17:56,017 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-10-29 15:17:56,018 INFO sqlalchemy.engine.Engine [cached since 0.1658s ago] ((50, 101.78913200000001, 101.78913200000001, 101.78913200000001, 152892.92499700002, 122592.662277), (51, 101.389132, 101.389132, 101.389132, 152892.92499700002, 122592.662277), (52, 101.28913200000001, 101.28913200000001, 101.28913200000001, 152892.92499700002, 122592.662277), (53, 100.889132, 100.889132, 100.889132, 152892.92499700002, 122592.662277), (54, 100.089132, 100.089132, 100.089132, 152892.92499700002, 122592.662277), (55, 99.689132, 99.689132, 99.689132, 152892.92499700002, 122

intervals for lithology======== [Interval({'top': Position({'middle': 0.0, 'upper': 0.0, 'lower': 0.0, 'units': 'm'}), 'base': Position({'middle': 0.1, 'units': 'm'}), 'description': '', 'data': {}, 'components': [Component({'lithology': 'not_exist'})]})]
Borehole geometry created successfully !
2021-10-29 15:17:56,067 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, date, length, diameter) VALUES (?, ?, ?, ?)
2021-10-29 15:17:56,068 INFO sqlalchemy.engine.Engine [cached since 0.2185s ago] ('212', None, 2.9999999999999982, 0.1)
2021-10-29 15:17:56,069 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-10-29 15:17:56,070 INFO sqlalchemy.engine.Engine [cached since 0.2178s ago] ((68, 101.929759, 101.929759, 101.929759, 152882.850194, 122591.453476), (69, 101.429759, 101.429759, 101.429759, 152882.850194, 122591.453476), (70, 101.129759, 101.129759, 101.129759, 152882.850194, 122591.453476), (71, 100.729759, 100.

...

2021-10-29 15:17:57,637 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, type, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-10-29 15:17:57,637 INFO sqlalchemy.engine.Engine [cached since 1.784s ago] ((186, 'F23', 0, 'sample', '{"benz\\u00e8ne": "vs"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "vr"}, {"cuivre": "vr"}, {"nickel": "vr"}', 372, 373, None), (187, 'F23', 1, 'sample', '{"benz\\u00e8ne": "vi"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "vr"}, {"cuivre": "vr"}, {"nickel": "vr"}', 374, 375, None))
2021-10-29 15:17:57,641 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  40
2021-10-29 15:17:57,644 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-29 15:17:57,645 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Borehole

Borehole geometry created successfully !
2021-10-29 15:17:57,705 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, date, length, diameter) VALUES (?, ?, ?, ?)
2021-10-29 15:17:57,706 INFO sqlalchemy.engine.Engine [cached since 1.857s ago] ('F25', None, 3.0, 0.1)
2021-10-29 15:17:57,707 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-10-29 15:17:57,707 INFO sqlalchemy.engine.Engine [cached since 1.855s ago] ((378, 102.369, 102.369, 102.369, 152872.429, 122579.02), (379, 101.869, 101.869, 101.869, 152872.429, 122579.02), (380, 99.369, 99.369, 99.369, 152872.429, 122579.02), (381, 98.869, 98.869, 98.869, 152872.429, 122579.02), (382, 97.169, 97.169, 97.169, 152872.429, 122579.02), (383, 96.669, 96.669, 96.669, 152872.429, 122579.02))
2021-10-29 15:17:57,708 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, type, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?,

2021-10-29 15:17:57,750 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, type, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-10-29 15:17:57,751 INFO sqlalchemy.engine.Engine [cached since 1.897s ago] ((192, 'F26', 0, 'sample', '{"benz\\u00e8ne": "vi"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vs"}, {"arsenic": "vs"}, {"cuivre": "vr"}, {"nickel": "vr"}', 384, 385, None), (193, 'F26', 1, 'sample', '{"benz\\u00e8ne": "vi"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "inconnu"}, {"cuivre": "vr"}, {"nickel": "inconnu"}', 386, 387, None), (194, 'F26', 2, 'sample', '{"benz\\u00e8ne": "inconnu"}, {"tolu\\u00e8ne": "inconnu"}, {"plomb": "inconnu"}, {"arsenic": "inconnu"}, {"cuivre": "inconnu"}, {"nickel": "inconnu"}', 388, 389, None), (195, 'F26', 3, 'sample', '{"benz\\u00e8ne": "inconnu"}, {"tolu\\u00e8ne": "inconnu"}, {"plomb": "inconnu"}, {"arsenic": "inconnu"}, {"cuivre": "inconnu"}, {"nickel": "inconnu"}', 390, 391, N

Borehole geometry created successfully !
2021-10-29 15:17:57,796 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, date, length, diameter) VALUES (?, ?, ?, ?)
2021-10-29 15:17:57,797 INFO sqlalchemy.engine.Engine [cached since 1.948s ago] ('F27a', None, 1.0, 0.1)
2021-10-29 15:17:57,798 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-10-29 15:17:57,799 INFO sqlalchemy.engine.Engine [cached since 1.947s ago] ((392, 102.773, 102.773, 102.773, 152877.366, 122593.917), (393, 102.273, 102.273, 102.273, 152877.366, 122593.917))
2021-10-29 15:17:57,800 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, type, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-10-29 15:17:57,800 INFO sqlalchemy.engine.Engine [cached since 1.573s ago] (196, 'F27a', 0, 'sample', '{"benz\\u00e8ne": "vi"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "vr"}, {"cuivre": "

Borehole geometry created successfully !


In [28]:
p.add_link_components_intervals(link_intv_comp)

2021-10-29 15:17:57,982 INFO sqlalchemy.engine.Engine INSERT INTO "Linkintervalcomponent" (intv_id, comp_id, extra_data) VALUES (?, ?, ?)
2021-10-29 15:17:57,983 INFO sqlalchemy.engine.Engine [generated in 0.00222s] ((0, 0, ''), (1, 0, ''), (2, 1, ''), (2, 2, ''), (2, 3, ''), (2, 4, ''), (2, 5, ''), (2, 6, '')  ... displaying 10 of 760 total bound parameter sets ...  (198, 16, ''), (198, 17, ''))
2021-10-29 15:17:57,998 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  46
2021-10-29 15:17:58,003 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-29 15:17:58,004 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-10-29 15:17:58,004 INFO sqlalchemy.engine.Engine [cached since 2.182s ago] ()


#### Validate modifications 

In [29]:
p.commit()

2021-10-29 15:17:58,010 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  46


In [30]:
p.refresh()

2021-10-29 15:17:58,019 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-29 15:17:58,020 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-10-29 15:17:58,021 INFO sqlalchemy.engine.Engine [cached since 2.199s ago] ()


In [31]:
session.close()

2021-10-29 15:17:58,029 INFO sqlalchemy.engine.Engine ROLLBACK


## update project legend dict

In [32]:
p.update_legend_cmap(compute_all_attrib=True, verbose=False)

In [33]:
p.legend_dict['lithology']['legend']

Legend(Decor({'_colour': '#ffffff', 'width': 3, 'hatch': None, 'component': Component({'lithology': 'not_exist'})}))

## Visualization

In [34]:
n = 1 # borehole index in the project's boreholes list
# df.query(f'ID=="{p.boreholes_3d[n].name}"')[['ID', 'Top_intv', 'Base_intv', 'Lithology']]
df.query(f'ID=="{p.boreholes_3d[n].name}"')

,ID,X,Y,Z,Long_for,Top_intv,Base_intv,Epais_intv,ID_ech,Litho_top,Litho_base,Lithology,Type_ech,Bnz,Toln,Pb,As,Cu,Ni
2,205,152887.693175,122594.619977,101.804167,4.8,0.2,0.6,0.4,205/0,0.0,0.2,Couche de boue en surface liée à l'interventio...,Sol,0.69,0.47,94.0,18.0,37.0,280.0
3,205,152887.693175,122594.619977,101.804167,4.8,1.9,2.4,0.5,205/1,0.0,0.2,Couche de boue en surface liée à l'interventio...,Sol,3.10,1.50,590.0,13.0,19.0,30.0
4,205,152887.693175,122594.619977,101.804167,4.8,3.0,3.2,0.2,205/3,0.0,0.2,Couche de boue en surface liée à l'interventio...,Sol,3.00,0.61,46.0,7.0,13.0,18.0
5,205,152887.693175,122594.619977,101.804167,4.8,3.2,3.6,0.4,205/3b,0.0,0.2,Couche de boue en surface liée à l'interventio...,Sol,0.80,0.13,12.0,10.0,9.6,22.0


In [35]:
attributes = list(legend_dict.keys())
attributes

['lithology', 'benzène', 'toluène', 'plomb', 'arsenic', 'cuivre', 'nickel']

### Log plot for one borehole

In [36]:
at = 1
n = 1
repr_att = attributes[at]

for i in p.boreholes_3d[n].intervals:
    for c in i.components:
        if hasattr(c, repr_att): print(i.top.middle, i.base.middle, c)

0.0 0.1 {'benzène': 'Inconnu'}


In [37]:
p.boreholes_3d[n]._intervals_dict

{'lithology': [Interval({'top': Position({'middle': 0.0, 'upper': 0.0, 'lower': 0.0, 'units': 'm'}), 'base': Position({'middle': 0.1, 'units': 'm'}), 'description': '', 'data': {}, 'components': [Component({'lithology': 'not_exist'}), Component({'benzène': 'Inconnu'}), Component({'toluène': 'Inconnu'}), Component({'plomb': 'Inconnu'}), Component({'arsenic': 'Inconnu'}), Component({'cuivre': 'Inconnu'}), Component({'nickel': 'Inconnu'})]})],
 'sample': [Interval({'top': Position({'middle': 101.604167, 'x': 152887.69317500002, 'y': 122594.619977, 'units': 'm'}), 'base': Position({'middle': 101.20416700000001, 'x': 152887.69317500002, 'y': 122594.619977, 'units': 'm'}), 'description': '{"benz\\u00e8ne": "vs"}, {"tolu\\u00e8ne": "vr"}, {"plomb": "vr"}, {"arsenic": "vr"}, {"cuivre": "vr"}, {"nickel": "vs"}', 'data': {'type': 'sample', 'sample_ID': 'test'}, 'components': [Component({'benzène': 'vs'}), Component({'toluène': 'vr'}), Component({'plomb': 'vr'}), Component({'arsenic': 'vr'}), Com

In [38]:
pause

NameError: name 'pause' is not defined

In [ ]:
p.boreholes_3d[n].plot_log(repr_attribute=repr_att)

### 3D display of one borehole

In [ ]:
p.boreholes_3d[n].plot_3d(repr_attribute=repr_att)

### Localization of project boreholes

### 3D display of project boreholes

In [ ]:
pause

In [ ]:
p.plot_3d(repr_attribute=repr_att, labels_size=15, bg_color=["royalblue", "aliceblue"], 
         window_size=(800, 600), verbose=True)